In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install keras-core wurlitzer

In [ ]:
!pip install "dill<0.3.2,>=0.3.1.1"
!pip install -U "numpy<1.25.0,>=1.14.3"
!pip install --upgrade --quiet  langchain langchain-community

In [ ]:
!pip install -U /kaggle/input/langchain-moonshot-0-0-6/langchain_moonshot-0.0.6-py3-none-any.whl

In [14]:
os.environ["MOONSHOT_API_KEY"] = 'sk-l6Wg4H......qw1N6v'
os.environ['MOONSHOT_API_BASE'] = "https://api.moonshot.cn/v1"

# Play chat

In [ ]:
import os
import langchain
from langchain_moonshot import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model_name="moonshot-v1-8k")
print('API_BASE: %s' % llm.openai_api_base)
print('PROXY: %s' % llm.openai_proxy)
print('API_KEY: %s' % llm.openai_api_key)
print('Model: %s' % llm.model_name)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

output_parser = StrOutputParser()

chat_chain = prompt | llm | output_parser

resp = chat_chain.invoke({"input": "how can langsmith help with testing?"})
print(resp)


# Play documents

In [ ]:
from langchain_moonshot import OpenAIClone
from pathlib import Path
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

client = OpenAIClone()

file_object = client.files.create(file=Path("/kaggle/input/optimizers/Deep Optimizers.pdf"), purpose="file-extract")
file_content = client.files.content(file_id=file_object.id).text
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents([Document(file_content)])
[print(document) for document in documents]

Set up vector database

In [9]:
from langchain_moonshot import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

from langchain_community.vectorstores import FAISS
# vector = FAISS.from_documents(documents, embeddings)  # TODO Investigate its logic

Chain multiple documents

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:

    <context>
    {context}
    </context>
    
    Question: {input}
    """
    )

# Create a chain for passing a list of Documents to a model.
document_chain = create_stuff_documents_chain(llm, prompt)  # llm | prompt

Summarize documents

In [ ]:
from langchain_core.documents import Document

summary = document_chain.invoke(
    {
        "input": "What about the types of optimizers?",
        "context": [Document(page_content="Many optimizers are available for us.")]
    }
)
print(summary)